In [1]:
%%capture
!pip install sentencepiece
!pip install transformers 

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
model.cuda();
model.eval();

In [ ]:
def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(
        text,
        return_tensors='pt',
        padding=True
        ).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(
        **x,
        encoder_no_repeat_ngram_size=grams,
        num_beams=beams,
        max_length=max_size,
        do_sample=do_sample
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [15]:
!wget https://raw.githubusercontent.com/shitkov/news_classification/main/titles.csv

--2021-08-31 13:41:14--  https://raw.githubusercontent.com/shitkov/news_classification/main/titles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607248 (593K) [text/plain]
Saving to: ‘titles.csv.2’

titles.csv.2        100%[===================>] 593.02K  --.-KB/s    in 0.03s   

2021-08-31 13:41:14 (21.1 MB/s) - ‘titles.csv.2’ saved [607248/607248]



In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('/content/titles.csv', index_col=0)

In [9]:
titles = list(data[data['label'] == 1]['title'])

In [18]:
from tqdm import tqdm

In [20]:
epoch = 4
new_titles = []
for _ in range(epoch):
    for t in tqdm(titles):
         new_titles.append(paraphrase(t, do_sample=True))

100%|██████████| 329/329 [02:11<00:00,  2.51it/s]


In [23]:
# deduplicate
new_titles = list(set(new_titles))

In [24]:
# save new dataset
headlines = pd.DataFrame()
headlines['title'] = new_titles
headlines['label'] = 1

In [27]:
# prepare data for concatination
data = data.drop(columns = ['sentence'])

headlines = pd.concat([headlines, data])

headlines.to_csv('/content/headlines.csv' ,index=False)